In [4]:
import numpy as np
import pandas as pd
import os

In [3]:
os.getcwd()

'/Users/lunal.brown/Personal/DataScience/github/decision_tree_easy'

In [11]:
df_w=pd.read_csv('weather_data_HanedaJan23rd2022.csv', header=0, encoding='cp932')
df_w.columns=np.arange(0, df_w.shape[1], 1)
df_w=df_w[[0,1,2,3,4,5,6]]
df_w=df_w[5:].reset_index(drop=True)
df_w=df_w.rename(columns={0:'datetime', 1:'temperature', 2:'high', 3:'low', 4:'wind', 5:'quolity', 6:'direction'})
df_w

,datetime,temperature,high,low,wind,quolity,direction
0,2022/1/14 1:00,3.9,8,1,7.2,8,北西
1,2022/1/14 2:00,2.4,8,1,5.5,8,北西
2,2022/1/14 3:00,2.2,8,1,7.5,8,北西
3,2022/1/14 4:00,2.6,8,1,6.8,8,北北西
4,2022/1/14 5:00,2.8,8,1,8.7,8,北北西
...,...,...,...,...,...,...,...
211,2022/1/22 20:00,7.8,8,1,0.5,8,東南東
212,2022/1/22 21:00,7.5,8,1,4.2,8,東北東
213,2022/1/22 22:00,6.9,8,1,3.5,8,北東
214,2022/1/22 23:00,6.8,8,1,3.5,8,北


In [9]:
df_w[:17]

,ダウンロードした時刻：2022/01/23 09:41:15,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,羽田,羽田,羽田,羽田,羽田,羽田,羽田,羽田,羽田,羽田,羽田
2,年月日時,気温(℃),気温(℃),気温(℃),風速(m/s),風速(m/s),風速(m/s),風速(m/s),風速(m/s),相対湿度(％),相対湿度(％),相対湿度(％)
3,NaN,NaN,NaN,NaN,NaN,NaN,風向,風向,NaN,NaN,NaN,NaN
4,NaN,NaN,品質情報,均質番号,NaN,品質情報,NaN,品質情報,均質番号,NaN,品質情報,均質番号
5,2022/1/14 1:00,3.9,8,1,7.2,8,北西,8,1,NaN,0,1
6,2022/1/14 2:00,2.4,8,1,5.5,8,北西,8,1,NaN,0,1
7,2022/1/14 3:00,2.2,8,1,7.5,8,北西,8,1,NaN,0,1
8,2022/1/14 4:00,2.6,8,1,6.8,8,北北西,8,1,NaN,0,1
9,2022/1/14 5:00,2.8,8,1,8.7,8,北北西,8,1,NaN,0,1
